In [ ]:
# fire_detection_training.py

import os
import zipfile
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import EfficientNetB3
import tensorflow.keras.backend as K

# ===========================
# 1. Kaggle Setup and Dataset Download
# ===========================
def setup_kaggle_and_download():
    from google.colab import files

    # Upload kaggle.json
    uploaded = files.upload()

    # Create .kaggle directory and move kaggle.json there
    os.makedirs('/root/.kaggle', exist_ok=True)
    os.rename('kaggle.json', '/root/.kaggle/kaggle.json')
    os.chmod('/root/.kaggle/kaggle.json', 0o600)

    # Install Kaggle package
    !pip install -q kaggle

    # Download dataset from Kaggle
    !kaggle datasets download -d christofel04/fire-detection-dataset

    # Extract dataset
    zip_path = '/content/fire-detection-dataset.zip'
    extract_to = '/content'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    print(f"Dataset extracted to {extract_to}")
    print("Extracted contents:")
    print(os.listdir(extract_to))

# ===========================
# 2. Data Generators Setup
# ===========================
def create_data_generators(train_dir, val_dir, test_dir, target_size=(224,224), batch_size=32):
    # Training data augmentation + rescale
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Validation and test data only rescale
    val_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False  # Important for evaluation
    )

    return train_generator, val_generator, test_generator

# ===========================
# 3. SqueezeNet Model Definition
# ===========================
def SqueezeNetEnhanced(input_shape=(224, 224, 3), num_classes=2):
    inputs = Input(shape=input_shape)

    # Conv1
    x = layers.Conv2D(96, (7, 7), strides=2, padding='valid', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=2)(x)

    def fire_module(x, squeeze_filters, expand_filters):
        squeeze = layers.Conv2D(squeeze_filters, (1, 1))(x)
        squeeze = layers.BatchNormalization()(squeeze)
        squeeze = layers.ReLU()(squeeze)
        expand1x1 = layers.Conv2D(expand_filters * 2, (1, 1), activation='relu')(squeeze)
        expand3x3 = layers.Conv2D(expand_filters * 2, (3, 3), padding='same', activation='relu')(squeeze)
        return layers.concatenate([expand1x1, expand3x3])

    x = fire_module(x, 16, 64)
    x = layers.Dropout(0.3)(x)
    x = fire_module(x, 16, 64)
    x = fire_module(x, 32, 128)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=2)(x)

    x = fire_module(x, 32, 128)
    x = layers.Dropout(0.3)(x)
    x = fire_module(x, 48, 192)
    x = fire_module(x, 48, 192)
    x = fire_module(x, 64, 256)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=2)(x)

    x = layers.Conv2D(num_classes, (1, 1))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Activation('softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# ===========================
# 4. Model Training and Evaluation for SqueezeNet
# ===========================
def train_and_evaluate_squeezenet(train_generator, val_generator, test_generator, epochs=10):
    model = SqueezeNetEnhanced(input_shape=(224, 224, 3), num_classes=2)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator
    )

    model.save_weights("squeezenet_fire_weights.weights.h5")

    # Evaluate on test data
    loss, accuracy = model.evaluate(test_generator)
    print(f"SqueezeNet Test Accuracy: {accuracy:.4f}")

    # Predictions for metrics
    y_true = test_generator.classes
    y_pred = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)

    precision = precision_score(y_true, y_pred_classes, average='weighted')
    recall = recall_score(y_true, y_pred_classes, average='weighted')
    f1 = f1_score(y_true, y_pred_classes, average='weighted')

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred_classes, target_names=list(test_generator.class_indices.keys())))

    cm = confusion_matrix(y_true, y_pred_classes)
    print("\nConfusion Matrix:")
    print(cm)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=test_generator.class_indices.keys(),
                yticklabels=test_generator.class_indices.keys())
    plt.title('Confusion Matrix - SqueezeNet')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

# ===========================
# 5. Focal Loss Definition for Class Imbalance
# ===========================
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return K.mean(K.sum(loss, axis=-1))
    return focal_loss_fixed

# ===========================
# 6. EfficientNetB3 Transfer Learning Model Creation
# ===========================
def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=2):
    base_model = EfficientNetB3(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

# ===========================
# 7. Transfer Learning Training with Focal Loss and Fine-tuning
# ===========================
def train_efficientnet_with_focal_loss(train_dir, val_dir, test_dir, epochs=50, batch_size=32):
    # Data generators with validation split (but since we use separate test_dir, val_dir should be validation set)
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1./255
    )

    train_gen = datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='


In [ ]:
val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights for imbalance
labels = train_gen.classes
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = dict(enumerate(class_weights))

model = create_efficientnet_model(input_shape=(224, 224, 3), num_classes=2)

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=focal_loss(alpha=0.25, gamma=2),
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    class_weight=class_weights_dict,
    callbacks=[reduce_lr, early_stop]
)

# Unfreeze top layers for fine-tuning
base_model = model.layers[0]
base_model.trainable = True

# Freeze lower layers, unfreeze upper layers (e.g., last 30 layers)
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=focal_loss(alpha=0.25, gamma=2),
    metrics=['accuracy']
)

history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs // 2,
    class_weight=class_weights_dict,
    callbacks=[reduce_lr, early_stop]
)

model.save_weights("efficientnetb3_focal_finetuned.weights.h5")

# Evaluate
loss, accuracy = model.evaluate(test_gen)
print(f"EfficientNetB3 Test Accuracy: {accuracy:.4f}")

y_true = test_gen.classes
y_pred = model.predict(test_gen)
y_pred_classes = np.argmax(y_pred, axis=1)

precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=list(test_gen.class_indices.keys())))

cm = confusion_matrix(y_true, y_pred_classes)
print("\nConfusion Matrix:")
print(cm)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_gen.class_indices.keys(),
            yticklabels=test_gen.class_indices.keys())
plt.title('Confusion Matrix - EfficientNetB3 Fine-tuned')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


In [ ]:
# Set your dataset directories here after extracting
train_dir = "/content/fire-detection-dataset/train"
val_dir = "/content/fire-detection-dataset/val"
test_dir = "/content/fire-detection-dataset/test"

# Create data generators
train_gen, val_gen, test_gen = create_data_generators(train_dir, val_dir, test_dir)

# Train and evaluate SqueezeNet
train_and_evaluate_squeezenet(train_gen, val_gen, test_gen, epochs=10)

# Train and evaluate EfficientNetB3 with focal loss and fine tuning
train_efficientnet_with_focal_loss(train_dir, val_dir, test_dir, epochs=50)



---

### Notes:
- Replace paths `train_dir`, `val_dir`, and `test_dir` with your actual directories.
- The Kaggle setup function is only needed if you want to automate dataset download inside Colab.
- Training epochs are set to modest defaults, you can increase as needed.
- Save weights only; you can save the full model with `model.save()` if preferred.
- Evaluation prints detailed metrics and plots confusion matrix.

